In [3]:
# import dask.dataframe as dd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StructType
from pyspark.sql.functions import unix_timestamp, from_unixtime, col, udf
from pyspark.sql.types import DateType
from datetime import datetime
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
sc = SparkContext('local')
spark = SparkSession(sc)
import pandas as pd
# from dataprep.clean import validate_country
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *


# In[4]:





22/01/25 00:27:46 WARN Utils: Your hostname, rahul-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.10.100 instead (on interface wlp8s0)
22/01/25 00:27:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/25 00:27:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df1 = spark.read.format("csv").option("header", "true").load("/home/rahul/Interview/Weatherforcast/city_temperature_sample.csv")
df2 = spark.read.format("csv").option("header", "true").load("/home/rahul/Interview/Weatherforcast/worldcities_sample.csv")
# df1.count()
# df2.count()


#Type cast day month and year to int 
df1=df1.withColumn('day',df1['day'].cast("int").alias('day'))
df1=df1.withColumn('month',df1['month'].cast("int").alias('month'))
df1=df1.withColumn('year',df1['year'].cast("int").alias('year'))
df1=df1.withColumn('AvgTemperature',df1['AvgTemperature'].cast("float").alias('AvgTemperature'))
df2=df2.withColumn('lat',df2['lat'].cast("float").alias('lat'))
df2=df2.withColumn('lng',df2['lng'].cast("float").alias('lng'))

df1.createOrReplaceTempView("city_temperature")
df2.createOrReplaceTempView("worldcities")


In [3]:

city_temperature1=spark.sql("""
select Region, country as temp_country, State, City as city_temp,
case when day >31 or day <1 then null else day end as day1, day,case when month >12 or month <1 then null else month end as month1,
month,year, case when length(year) <4 or length(year) >4 then null else year end year1,
AvgTemperature from city_temperature """)

city_temperature1.createOrReplaceTempView("city_temperature1")



In [4]:
city_temperature1.show()

+------+------------+-----+---------+----+---+------+-----+----+-----+--------------+
|Region|temp_country|State|city_temp|day1|day|month1|month|year|year1|AvgTemperature|
+------+------------+-----+---------+----+---+------+-----+----+-----+--------------+
|Africa|       Japan| null|  Algiers|   1|  1|     1|    1|1995| 1995|          64.2|
|Africa|       Japan| null|  Algiers|   2|  2|     1|    1| 195| null|          49.4|
|Africa|       Japan| null|  Algiers|  21| 21|     3|    3|null| null|          56.7|
|Africa|       Japan| null|  Algiers|  22| 22|  null|   13|1995| 1995|          51.4|
|Africa|       Japan| null|  Algiers|null| 32|     3|    3|1995| 1995|          51.6|
|Africa|       Japan| null|  Algiers|null|  0|     3|    3|1995| 1995|          53.3|
|Africa|       Japan| null|  Algiers|  25| 25|  null|    0|1995| 1995|          56.4|
|Africa|       Japan| null|  Algiers|  26| 26|     3|    3|1995| 1995|          54.7|
|Africa|       Japan| null|  Algiers|  27| 27|     3| 

In [5]:
df3=spark.sql("""
select Region, temp_country, State,city_temp, CONCAT(Day1, '/',  Month1, '/',Year1) as date_temp,
day, month,year,
AvgTemperature  from city_temperature1 """)
broadcast_df=df3.join(broadcast(df2),df3.temp_country==df2.country).cache()

#remove null field date fields

# broadcast_df.na.drop(Seq("date_temp")).show(false)
broadcast_df=broadcast_df.where(broadcast_df.date_temp.isNotNull())

In [6]:
broadcast_df.show()

+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|   city|city_ascii|    lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+
|Africa|       Japan| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|
|Africa|       Japan| null|  Algiers|26/3/1995| 26|    3|1995|          54.7|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|
|Africa|       Japan| null|  Algiers|27/3/1995| 27|    3|1995|          54.5|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|    

In [7]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
func =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())

broadcast_df1 = broadcast_df.withColumn('date_new', func(col('date_temp')))
broadcast_df1.printSchema()
broadcast_df1.show()

root
 |-- Region: string (nullable = true)
 |-- temp_country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- city_temp: string (nullable = true)
 |-- date_temp: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- AvgTemperature: float (nullable = true)
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date_new: date (nullable = true)

+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|

In [8]:
##dropping a column from the dataframe different ways
# pyspark.sql.functions import col

broadcast_df1.drop("iso3").printSchema()

broadcast_df1.drop(col("iso2")) \
  .printSchema()  
  
broadcast_df1.drop(broadcast_df1.iso2) \
  .printSchema()

root
 |-- Region: string (nullable = true)
 |-- temp_country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- city_temp: string (nullable = true)
 |-- date_temp: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- AvgTemperature: float (nullable = true)
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date_new: date (nullable = true)

root
 |-- Region: string (nullable = true)
 |-- temp_country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- city_temp: string (nullable = true)
 |-- date_temp: string (nullable = true)
 |-- day: integ

In [9]:
#dropping more than 1 columns
broadcast_df1.drop("iso3","iso2") \
    .printSchema()

cols = ("iso3","iso2")

broadcast_df1.drop(*cols) \
   .printSchema()

root
 |-- Region: string (nullable = true)
 |-- temp_country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- city_temp: string (nullable = true)
 |-- date_temp: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- AvgTemperature: float (nullable = true)
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)
 |-- country: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date_new: date (nullable = true)

root
 |-- Region: string (nullable = true)
 |-- temp_country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- city_temp: string (nullable = true)
 |-- date_temp: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: int

In [10]:
##PySpark splits the records based on the partition column and stores each partition data into a sub-directory.
broadcast_df1.write.option("header",True) \
        .partitionBy("year") \
        .mode("overwrite") \
        .csv("/home/rahul/Interview/Weatherforcast/broadcast_df1.1.csv/year/")

In [11]:
##PySpark splits the records based on the partition column and stores each partition data into a sub-directory.
broadcast_df1.write.option("header",True) \
        .partitionBy("year","temp_country") \
        .mode("overwrite") \
        .csv("/home/rahul/Interview/Weatherforcast/broadcast_df1.1.csv/")

In [12]:
#Use repartition() and partitionBy() together
broadcast_df1.repartition(2) \
        .write.option("header",True) \
        .partitionBy("year") \
        .mode("overwrite") \
        .csv("/home/rahul/Interview/Weatherforcast/broadcast_df1.rep.csv/")

In [13]:
#partitionBy() control number of partitions
broadcast_df1.write.option("header",True) \
        .option("maxRecordsPerFile", 2) \
        .partitionBy("year") \
        .mode("overwrite") \
        .csv("/home/rahul/Interview/Weatherforcast/broadcast_df1.rep.csv/")

In [14]:
#pivoting of country
broadcast_df1.groupBy("year").pivot("Country").avg("AvgTemperature").show()


+----+-----------------+
|year|            Japan|
+----+-----------------+
|1995|55.81764692418716|
+----+-----------------+



In [15]:


#highest temperature  based on countries
w3 = Window.partitionBy("country").orderBy(col("AvgTemperature").desc())
broadcast_df1.withColumn("row",row_number().over(w3))   .filter(col("row") == 1).drop("row")   .show()


# In[10]:


broadcast_df1.withColumn("row",row_number().over(w3))   .withColumn("avg", avg(col("AvgTemperature")).over(w3))   .withColumn("sum", sum(col("AvgTemperature")).over(w3))   .withColumn("min", min(col("AvgTemperature")).over(w3))   .withColumn("max", max(col("AvgTemperature")).over(w3))   .where(col("row")==1).select("country","avg","sum","min","max")   .show()


# In[75]:



broadcast_df1.createOrReplaceTempView("broadcast_df1")
broadcast_df1.cache()
broadcast_df1.show()


# In[98]:


country_lkp = spark.read.option("header", "true").option("quote", "\"").option("escape", "\"").csv("/home/rahul/Interview/Weatherforcast/country_lkp.csv")
country_lkp.createOrReplaceTempView("country_lkp")
country_lkp.cache() 
country_lkp.show()


# In[91]:


spark.sql("""
select * from country_lkp where UPPER(country) like '%ARUBA%'""").show()


# In[100]:


broadcast_df2=spark.sql("""
select a.* from broadcast_df1 a inner join country_lkp b on upper(a.country)=upper(b.country) """)


# In[102]:



# broadcast_df2 = broadcast_df1.merge(country_lkp, left_on=broadcast_df1["country"].str.lower(), right_on=["country"].str.lower(), how="left")

broadcast_df2.createOrReplaceTempView("broadcast_df2")
broadcast_df2.cache()
broadcast_df2.show()





# #Total number of countries
no_of_cities=spark.sql("""select count(city_temp) as no_of_cities from broadcast_df2""")
no_of_cities.show()


+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|   city|city_ascii|    lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|  date_new|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|Africa|       Japan| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|1995-01-01|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+



+-------+-----------------+------------------+----+----+
|country|              avg|               sum| min| max|
+-------+-----------------+------------------+----+----+
|  Japan|64.19999694824219|128.39999389648438|64.2|64.2|
+-------+-----------------+------------------+----+----+

+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|   city|city_ascii|    lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|  date_new|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+
|Africa|       Japan| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1

In [16]:

# #Total number of countries
no_of_countries=spark.sql("""select count(temp_country) as no_of_countries from broadcast_df2""")
no_of_countries.cache()
no_of_countries.show()


# #AVg temp of around the world
avg_wrldtemp=spark.sql("""select avg(AvgTemperature) as avg_wrldtemp from broadcast_df2""")
avg_wrldtemp.cache()
avg_wrldtemp.show()


# #AVg temp of around the world
avg_cntrytemp=spark.sql("""select avg(AvgTemperature) as avg_cntrytemp from broadcast_df2 group by temp_country""")
avg_cntrytemp.cache()
avg_cntrytemp.show()

# #AVg temp of around the world between year 1995 and 2000
avg_cntrytemp_1995_2000=spark.sql("""select temp_country as countries,avg(AvgTemperature) as avg_cntrytemp from broadcast_df2 where year between 1995 and 2000 group by temp_country""")
avg_cntrytemp_1995_2000.cache()
avg_cntrytemp_1995_2000.show()

# #AVg temp of around the world between year 1995 and 2000
avg_cntrytemp_2000_2010=spark.sql("""select temp_country as countries, avg(AvgTemperature) as avg_cntrytemp from broadcast_df2 where year between 2000 and 2010 group by temp_country""")
avg_cntrytemp_2000_2010.cache()
avg_cntrytemp_2000_2010.show()

+---------------+
|no_of_countries|
+---------------+
|             17|
+---------------+

+-----------------+
|     avg_wrldtemp|
+-----------------+
|55.81764692418716|
+-----------------+

+-----------------+
|    avg_cntrytemp|
+-----------------+
|55.81764692418716|
+-----------------+

+---------+-----------------+
|countries|    avg_cntrytemp|
+---------+-----------------+
|    Japan|55.81764692418716|
+---------+-----------------+

+---------+-------------+
|countries|avg_cntrytemp|
+---------+-------------+
+---------+-------------+



In [17]:
# #AVg temp of around the world between year 1995 and 2000
avg_cntrytemp_2000_2010=spark.sql("""select * from broadcast_df2 where year between 2000 and 2010 limit 10""")
avg_cntrytemp_2000_2010.cache()
avg_cntrytemp_2000_2010.show()


+------+------------+-----+---------+---------+---+-----+----+--------------+----+----------+---+---+-------+----+----+----------+-------+----------+---+--------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|city|city_ascii|lat|lng|country|iso2|iso3|admin_name|capital|population| id|date_new|
+------+------------+-----+---------+---------+---+-----+----+--------------+----+----------+---+---+-------+----+----+----------+-------+----------+---+--------+
+------+------------+-----+---------+---------+---+-----+----+--------------+----+----------+---+---+-------+----+----+----------+-------+----------+---+--------+



In [19]:
data_distinct_state=spark.sql("""SELECT  *
FROM    (SELECT *,
                ROW_NUMBER() OVER (PARTITION BY state ORDER BY date_temp) AS RowNumber
         FROM   broadcast_df2
         ) AS a
WHERE   a.RowNumber = 1""").show()

+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+---------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|   city|city_ascii|    lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|  date_new|RowNumber|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+---------+
|Africa|       Japan| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|Tokyo12|     Tokyo|35.6897|139.6922|  Japan| JP1| JPN|     Tōkyō|primary|  37977000|1392685764|1995-01-01|        1|
+------+------------+-----+---------+---------+---+-----+----+--------------+-------+----------+-------+--------+-------+----+----+----------+-------+----------+----------+----------+---------+



In [4]:
distinct_original_1bil_records= spark.read.format("csv").option("header", "true").load("/home/rahul/Interview/Weatherforcast/broadcast_df.csv/data.csv")

In [28]:
distinct_original_1bil_records.count()

104049799

In [5]:
distinct_original_1bil_records.cache()
distinct_original_1bil_records.show()
distinct_original_1bil_records.createOrReplaceTempView("distinct_original_1bil_records")

+------+------------+-----+---------+---------+---+-----+----+--------------+---------------+---------------+-------+------+-------+----+----+------------------+-------+----------+----------+
|Region|temp_country|State|city_temp|date_temp|day|month|year|AvgTemperature|           city|     city_ascii|    lat|   lng|country|iso2|iso3|        admin_name|capital|population|        id|
+------+------------+-----+---------+---------+---+-----+----+--------------+---------------+---------------+-------+------+-------+----+----+------------------+-------+----------+----------+
|Africa|     Algeria| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|     I-n-Amguel|     I-n-Amguel|23.6936|5.1647|Algeria|  DZ| DZA|       Tamanrasset|   null|      3030|1012162135|
|Africa|     Algeria| null|  Algiers| 1/1/1995|  1|    1|1995|          64.2|      Tassamert|      Tassamert|36.2693| 4.823|Algeria|  DZ| DZA|Bordj Bou Arréridj|   null|      5269|1012240413|
|Africa|     Algeria| null|  Algiers| 1/

In [37]:
data_distinct_state=spark.sql("""SELECT  *
FROM    (SELECT *,
                ROW_NUMBER() OVER (PARTITION BY state ORDER BY date_temp) AS RowNumber
         FROM   distinct_original_1bil_records where temp_country='Algeria'
         ) AS a
WHERE   a.RowNumber = 1""")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/rahul/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rahul/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/rahul/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o22.sql

In [14]:
data_distinct_state=spark.sql("""SELECT  *
FROM (SELECT *,
                ROW_NUMBER() OVER (PARTITION BY temp_country ORDER BY date_temp) AS RowNumber
         FROM   distinct_original_1bil_records ) AS a
         where a.RowNumber =1 limit 10
        """)

In [15]:
data_distinct_state.cache()
data_distinct_state.show()

22/01/25 00:44:37 WARN MemoryStore: Not enough space to cache rdd_13_18 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:38 WARN MemoryStore: Not enough space to cache rdd_13_19 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:39 WARN MemoryStore: Not enough space to cache rdd_13_20 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:41 WARN MemoryStore: Not enough space to cache rdd_13_21 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:42 WARN MemoryStore: Not enough space to cache rdd_13_22 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:43 WARN MemoryStore: Not enough space to cache rdd_13_23 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:45 WARN MemoryStore: Not enough space to cache rdd_13_24 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:46 WARN MemoryStore: Not enough space to cache rdd_13_25 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:47 WARN MemoryStore: Not enough space to cache rdd_13_26 in memory! (computed 5.1 MiB so far)
22/01/25 00:44:48 W

22/01/25 00:46:14 WARN MemoryStore: Not enough space to cache rdd_13_93 in memory! (computed 4.1 MiB so far)
22/01/25 00:46:15 WARN MemoryStore: Not enough space to cache rdd_13_94 in memory! (computed 4.1 MiB so far)
22/01/25 00:46:16 WARN MemoryStore: Not enough space to cache rdd_13_95 in memory! (computed 4.1 MiB so far)
22/01/25 00:46:17 WARN MemoryStore: Not enough space to cache rdd_13_96 in memory! (computed 4.1 MiB so far)
22/01/25 00:46:19 WARN MemoryStore: Not enough space to cache rdd_13_97 in memory! (computed 4.1 MiB so far)


[1322.770s][warning][gc,alloc] Executor task launch worker for task 59.0 in stage 17.0 (TID 930): Retried waiting for GCLocker too often allocating 131074 words


22/01/25 00:49:48 ERROR Executor: Exception in task 59.0 in stage 17.0 (TID 930)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:50)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:523)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:172)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at org.apache.spark.sql.execution.window.WindowExec$$anon$1.fetchNextRow(WindowExec.scala:133)
	at org.apa

Py4JJavaError: An error occurred while calling o42.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 59 in stage 17.0 failed 1 times, most recent failure: Lost task 59.0 in stage 17.0 (TID 930) (192.168.10.100 executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:50)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:523)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:172)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execu